In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the data
data = pd.read_csv('epl-training_basic.csv')

# Encode categorical variables
le_teams = LabelEncoder()
data['HomeTeam'] = le_teams.fit_transform(data['HomeTeam'])
data['AwayTeam'] = le_teams.fit_transform(data['AwayTeam'])

le_result = LabelEncoder()
data['Result'] = le_result.fit_transform(data['FTR'])

# Define features and target variable
X = data[['HomeTeam', 'AwayTeam']]
y = data['Result']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid.best_params_)

# Generate predictions for all combinations of teams
teams = le_teams.classes_
win_rates = []

for home_team, away_team in itertools.product(teams, repeat=2):
    home_team_enc = le_teams.transform([home_team])[0]
    away_team_enc = le_teams.transform([away_team])[0]
    match_features = scaler.transform([[home_team_enc, away_team_enc]])
    prediction = grid.predict(match_features)[0]
    win_rates.append((home_team, away_team, le_result.inverse_transform([prediction])[0]))

# Convert win rates to DataFrame for easier plotting
win_rates_df = pd.DataFrame(win_rates, columns=['HomeTeam', 'AwayTeam', 'Result'])

# Assign numerical values to results
win_rates_df['ResultValue'] = win_rates_df['Result'].apply(lambda x: 1 if x == 'H' else (0.5 if x == 'D' else 0))

# Plotting the 3D graph
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Encode teams for plotting
win_rates_df['HomeTeamEncoded'] = le_teams.transform(win_rates_df['HomeTeam'])
win_rates_df['AwayTeamEncoded'] = le_teams.transform(win_rates_df['AwayTeam'])

ax.scatter(win_rates_df['HomeTeamEncoded'], win_rates_df['AwayTeamEncoded'], win_rates_df['ResultValue'], c=win_rates_df['ResultValue'], cmap='viridis')

ax.set_xlabel('Home Team')
ax.set_ylabel('Away Team')
ax.set_zlabel('Predicted Result')

# Set tick labels
ax.set_xticks(range(len(le_teams.classes_)))
ax.set_xticklabels(le_teams.classes_, rotation=90)
ax.set_yticks(range(len(le_teams.classes_)))
ax.set_yticklabels(le_teams.classes_)

plt.show()